In [6]:
import torch
import torchvision
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import matplotlib.pyplot as plt
import random
import pandas as pd
import numpy as np
import os

In [3]:
import os
data_dir = 'C:/Users/User/Google 드라이브/AIFFEL/+Final project/data/'
data_path = os.path.join(data_dir, 'prep_data_5x5.csv')
data = pd.read_csv(data_path)
data = data.drop(['Unnamed: 0'], axis = 1)
data.head(10)

,FSR1,FSR2,FSR3,FSR4,FSR5,FSR9,FSR10,FSR11,FSR12,FSR13,...,FSR26,FSR27,FSR28,FSR29,FSR33,FSR34,FSR35,FSR36,FSR37,Label
0,5,2,0,1,2,1,16,35,49,24,...,10,8,0,0,4,6,4,0,5,1
1,2,1,0,3,0,1,4,33,42,26,...,13,2,0,0,4,0,0,1,6,2
2,6,0,7,10,0,4,3,37,37,25,...,8,5,3,5,0,0,4,0,0,3
3,0,2,2,2,0,0,0,34,44,26,...,3,8,1,0,0,7,1,0,0,4
4,0,2,6,11,3,0,1,30,41,23,...,7,9,2,0,3,7,0,0,0,5
5,0,0,5,9,3,0,5,33,42,26,...,6,7,0,0,0,6,2,2,1,6
6,0,7,1,4,0,0,0,32,42,21,...,9,7,0,4,2,10,1,3,2,7
7,0,2,2,11,5,1,0,37,41,15,...,14,5,0,0,0,0,5,0,0,8
8,2,4,4,12,4,3,0,31,35,24,...,14,2,0,0,4,5,3,5,0,9
9,0,5,6,3,3,3,3,34,42,18,...,12,9,0,0,4,5,3,0,1,10


In [4]:
feature = data.iloc[:,:-1]
label = data.iloc[:, -1]
print(feature.shape)
print(label.shape)

(300, 25)
(300,)


In [7]:
temp_array_feature = feature.to_numpy()

array_feature = []
for i in range(len(temp_array_feature)):
    reshaped_array = temp_array_feature[i].reshape(5,5)
    array_feature.append(reshaped_array)
    
array_feature = np.array(array_feature)
array_feature = array_feature.astype('uint8')
array_feature.shape

(300, 5, 5)

In [13]:
torch_feature = torch.from_numpy(array_feature)
array_label = label.to_numpy()
torch_label = torch.from_numpy(array_label)
print(torch_feature.shape)
print(torch_label.shape)

torch.Size([300, 5, 5])
torch.Size([300])


In [2]:
USE_CUDA = torch.cuda.is_available() # GPU를 사용가능하면 True, 아니라면 False를 리턴
device = torch.device("cuda" if USE_CUDA else "cpu") # GPU 사용 가능하면 사용하고 아니면 CPU 사용
print("다음 기기로 학습합니다:", device)

다음 기기로 학습합니다: cuda


In [3]:
# for reproducibility
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [8]:

nearest_mode = torchvision.transforms.InterpolationMode.NEAREST
normalize = torchvision.transforms.Normalize(mean = 0.5, std = 0.229)

In [14]:
# hyperparameters
training_epochs = 15
batch_size = 100
target_class_num = 10
image_size = 5

In [9]:
train_transformer = torchvision.transforms.Compose([
    torchvision.transforms.Resize((image_size,image_size), interpolation = nearest_mode),
#     torchvision.transforms.CenterCrop((224,224)),
    torchvision.transforms.ToTensor(),
    normalize
])

test_transformer = torchvision.transforms.Compose([
    torchvision.transforms.Resize((image_size,image_size), interpolation = nearest_mode),
    torchvision.transforms.ToTensor(),
    normalize
])

In [10]:
# # MNIST dataset
# mnist_train = dsets.MNIST(root='MNIST_data/',
#                           train=True,
#                           transform=train_transformer,
#                           download=True)

# mnist_test = dsets.MNIST(root='MNIST_data/',
#                          train=False,
#                          transform=test_transformer,
#                          download=True)

In [11]:
train_dataloader = torch.utils.data.DataLoader(mnist_train, batch_size = batch_size, shuffle = True)
test_dataloader = torch.utils.data.DataLoader(mnist_test, shuffle = False)

In [15]:
net = torchvision.models.resnet50(pretrained = True)
net.fc = torch.nn.Linear(
    net.fc.in_features,
    target_class_num
)

In [16]:
net.eval()

net.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [17]:
train_preds = list()
train_actvecs = list()
train_outputs_softmax = list()
train_labels = list()

with torch.no_grad():
  for idx, (img, label) in enumerate(train_loader):
      img = img.to(device)
      label = label.to(device)

      out = net(img)
      out_actvec = out.cpu().detach().numpy()[0]
      out_softmax = torch.softmax(out, 1).cpu().detach().numpy()[0]
      out_pred = int(torch.argmax(out).cpu().detach().numpy())
      out_label = int(label.cpu().detach().numpy())

      train_actvecs.append(out_actvec) # component 1: softmax 전의 Activation Vector
      train_preds.append(out_pred) # componenet 2: 각 데이터에 대한 예측값
      train_outputs_softmax.append(out_softmax) # component 3: 각 데이터에 대한 softmax 확률
      train_labels.append(out_label) # component 4: 각 데이터에 대한 Label (정답)

train_actvecs = np.asarray(train_actvecs)
train_preds = np.asarray(train_preds)
train_outputs_softmax = np.asarray(train_outputs_softmax)
train_labels = np.asarray(train_labels)

NameError: name 'train_loader' is not defined